## A\B Testing - does the page affect the students?

In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('../../datasets/checking-logs.sqlite')

#### Using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows

In [3]:
sql = """
WITH time_period_deltas AS (
    SELECT t.uid, 
    CASE
        WHEN first_commit_ts < first_view_ts THEN 'before'
        ELSE 'after'
    END as time_period,
    avg((strftime('%s', first_commit_ts) - d.deadlines)/3600) as avg_diff
    FROM test t
    JOIN deadlines d on labs = t.labname
    WHERE 
        labname != 'project1'
    GROUP BY 
        t.uid, time_period
), valid_users as (
    SELECT uid, COUNT(DISTINCT time_period) as valid_users
    FROM time_period_deltas
    GROUP BY uid
)
SELECT time_period, avg(avg_diff) as  avg_diff
FROM time_period_deltas t
JOIN valid_users v on t.uid = v.uid 
WHERE valid_users = 2
Group by time_period
"""
test_results = pd.read_sql(sql, con)
print(test_results)

  time_period   avg_diff
0       after -99.523810
1      before -66.047619


In [4]:
sql = """
WITH time_period_deltas AS (
    SELECT c.uid, 
    CASE
        WHEN first_commit_ts < first_view_ts THEN 'before'
        ELSE 'after'
    END as time_period,
    avg((strftime('%s', first_commit_ts) - d.deadlines)/3600) as avg_diff
    FROM control c
    JOIN deadlines d on labs = c.labname
    WHERE 
        labname != 'project1'
    GROUP BY 
        c.uid, time_period
), valid_users as (
    SELECT uid, COUNT(DISTINCT time_period) as valid_users
    FROM time_period_deltas
    GROUP BY uid
)
SELECT time_period, avg(avg_diff) as  avg_diff
FROM time_period_deltas t
JOIN valid_users v on t.uid = v.uid 
WHERE valid_users = 2
Group by time_period
"""
control_results = pd.read_sql(sql, con)
print(control_results)

  time_period   avg_diff
0       after -99.322222
1      before -98.033333


### Result - the page definitely affect the students

In [5]:
con.close()